In [ ]:
from models.llm import load_model, generate_comments, prepare_prompt

# Load model and tokenizer
model_name = "codellama/CodeLlama-7b-Instruct-hf"
model, tokenizer = load_model(model_name)

# Sample test input
test_input = {"prev_context": "def add(a, b):", "next_context": "    return a + b", "code": "def add(a, b):\n    return a + b"}

# Test different prompt strategies
prompt_types = ["zero-shot", "few-shot", "cot", "critique", "expert_prompt"]

for prompt_type in prompt_types:
    print(f"\nTesting {prompt_type} Prompt Strategy:")
    prompt = prepare_prompt(test_input, "inline", prompt_type=prompt_type)
    comments = generate_comments(model, tokenizer, prompt, prompt_type=prompt_type)
    print(f"Generated Comments:\n{comments}")